This script is designed to augment the [Open Assistant/oasst2](huggingface.co/datasets/OpenAssistant/oasst2) dataset by adding a CRITERIA score, which helps filter out irrelevant messages unrelated to veganism or animal rights.

The filtered dataset can then be used either as seed data for further human feedback collection or directly for model training.

The CRITERIA score evaluates eight key aspects of the content:

*   Cultural Sensitivity (C): Respect for diverse cultural perspectives in animal advocacy.
*   Relevance (R): Pertinence of the content to veganism and animal rights.
*  Insight (I): Level of original insights provided by the content.
*  Trustworthiness (T): Accuracy, reliability, and credibility of the information.
*  Emotional Impact (E): Effectiveness in eliciting empathy and emotional
 engagement.
*  Rationality (R): Logical consistency and reasoning in the content.
* Influence (I): Potential of the content to encourage actions and lifestyle changes.
* Alignment (A): Consistency with vegan and animal rights ethics.

Running this script requires an account with permissions to a Google Cloud project that has Vertex AI's API enabled and also requires the oaast2 dataset being placed inside that account's Google Drive.

In [ ]:
# Install and import necessary libraries - Google Cloud AI Platform, Google Auth, and Requests
!pip install google-cloud-aiplatform google-auth requests

import json
import re
import google.auth
from google.colab import auth
from google.cloud import aiplatform
from google.colab import drive
from vertexai.generative_models import GenerativeModel, Content, Part, GenerationConfig
from google.colab import files

# Authenticate to Google Cloud - this will open a new tab to authenticate
print("Authenticating to Google Cloud...")
auth.authenticate_user()

# Mount Google Drive to access files stored in the drive - this will open a new tab to authenticate
print("Mounting Google Drive...")
drive.mount('/content/drive')

# Set up your Google Cloud project and location - replace with your actual project ID and location
PROJECT_ID = 'PROJECT_ID'  # Replace with your actual project ID - e.g., 'my-project-id'
LOCATION = 'LOCATION'    # Replace with the Google Cloud region you want to use - e.g., 'us-central1'

# Initialize Vertex AI with the specified project and location - this will set the default project and location for Vertex AI
print(f"Initializing Vertex AI for project {PROJECT_ID} in location {LOCATION}...")
aiplatform.init(project=PROJECT_ID, location=LOCATION)

# Define the model ID for the Gemini model to be used for generating content - replace with your actual model ID
MODEL_ID = 'gemini-1.5-flash-001'

def extract_json_from_response(response_text):
    """
    Extract JSON object from the response text.

    Args:
    response_text (str): The response text from which to extract JSON.

    Returns:
    dict or None: The extracted JSON object or None if extraction fails.
    """
    try:
        match = re.search(r'{.*}', response_text, re.DOTALL)
        if match:
            return json.loads(match.group(0))
    except json.JSONDecodeError:
        print(f"Failed to parse JSON response: {response_text}")
    return None

def rank_message(message_text):
    """
    Rank a message based on the CRITERIA scale using the Gemini model.

    Args:
    message_text (str): The text of the message to be ranked.

    Returns:
    dict or None: A dictionary containing CRITERIA scores or None if ranking fails.
    """
    prompt = f"""
    You will be tasked with evaluating content based on the CRITERIA scale. For each piece of content provided, you will generate scores for the following eight criteria, each on a scale from 0 to 1. Output the results in JSON format.

    ### Criteria

    1. **Cultural Sensitivity**: Measure how culturally inclusive the content is.
       - **Culturally Inclusive (0.8-1.0)**: The content shows respect for diverse cultural perspectives and uses culturally sensitive approaches to animal advocacy.
       - **Moderately Inclusive (0.4-0.7)**: The content generally respects cultural diversity but may lack depth in cultural sensitivity.
       - **Culturally Insensitive (0.0-0.3)**: The content lacks respect for cultural diversity and fails to use culturally sensitive approaches.
    2. **Relevance**: Measure how pertinent the content is to veganism and animal rights.
       - **Highly Relevant (0.8-1.0)**: The content is directly related to veganism, animal rights, vegan lifestyle, plant-based diets, animal welfare, ethical treatment of animals, or advocacy for animal rights.
       - **Moderately Relevant (0.4-0.7)**: The content indirectly relates to veganism and animal rights through broader ethical, dietary, or sustainability discussions.
       - **Not Relevant (0.0-0.3)**: The content is unrelated to veganism or animal rights.
    3. **Insight**: Judge the level of insight provided by the key concept in the content.
       - **Highly Insightful (0.8-1.0)**: The content provides deep, original insights that significantly advance the understanding of veganism or animal advocacy.
       - **Moderately Insightful (0.4-0.7)**: The content offers useful insights that enhance understanding but may not be particularly original.
       - **No Unique Insights (0.0-0.3)**: The content provides no meaningful insights or repeats well-known information.
    4. **Trustworthiness**: Rate the accuracy, reliability, and credibility of the information presented.
       - **Highly Trustworthy (0.8-1.0)**: The information is accurate, well-researched, and comes from credible sources.
       - **Moderately Trustworthy (0.4-0.7)**: The information is generally accurate but may include some minor errors or questionable sources.
       - **Untrustworthy (0.0-0.3)**: The information is inaccurate, misleading, or based on non-credible sources.
    5. **Emotional Impact**: Measure the emotional engagement the content provides.
       - **Very Emotionally Impactful (0.8-1.0)**: The content effectively elicits empathy and emotional engagement.
       - **Moderately Emotionally Impactful (0.4-0.7)**: The content elicits some emotional engagement but may lack depth.
       - **Not Emotionally Impactful (0.0-0.3)**: The content fails to elicit any emotional response.
    6. **Rationality**: Evaluate the logical consistency and reasoning in the content.
       - **Very Rational (0.8-1.0)**: The content is logically consistent, well-reasoned, and supported by evidence.
       - **Moderately Rational (0.4-0.7)**: The content is generally rational but may contain some logical inconsistencies or weak arguments.
       - **Not Rational (0.0-0.3)**: The content lacks logical consistency and sound reasoning.
    7. **Influence**: Assess the potential of the content to encourage actions and lifestyle changes.
       - **Highly Likely to Influence Behavior (0.8-1.0)**: The content has strong potential to encourage actions and lifestyle changes.
       - **Moderately Likely to Influence Behavior (0.4-0.7)**: The content has some potential to influence behavior but may not be compelling enough to drive significant changes.
       - **Not Likely to Influence Behavior (0.0-0.3)**: The content is unlikely to influence any behavior change.
    8. **Alignment**: Assess how well the content aligns with vegan and animal rights ethics.
       - **Highly Aligned (0.8-1.0)**: The content strongly aligns with the ethical principles and core values of veganism and animal rights.
       - **Moderately Aligned (0.4-0.7)**: The content supports some aspects of vegan ethics but may include neutral or slightly contradictory elements.
       - **Not Aligned (0.0-0.3)**: The content contradicts or is indifferent to vegan principles.

    ### JSON Output Format

    For each piece of content, output a JSON object with the following structure:

    {{
      "CRITERIA_scores": {{
        "Cultural_Sensitivity": <score_from_0_to_1>,
        "Relevance": <score_from_0_to_1>,
        "Insight": <score_from_0_to_1>,
        "Trustworthiness": <score_from_0_to_1>,
        "Emotional_Impact": <score_from_0_to_1>,
        "Rationality": <score_from_0_to_1>,
        "Influence": <score_from_0_to_1>,
        "Alignment": <score_from_0_to_1>
      }},
      "CRITERIA_final_score": <average_of_all_scores>
    }}

    Content:
    {message_text}
    """

    try:
        # Create the GenerativeModel object - this will load the model for generating content
        gemini_model = GenerativeModel(model_name=MODEL_ID)

        # Set up the generation configuration with parameters controlling the output - adjust as needed
        generation_config = GenerationConfig(
            temperature=0.7,          # Controls the randomness of the output - higher values make the output more random
            max_output_tokens=512,    # Maximum number of tokens in the output - adjust based on the model's maximum output length
            top_p=0.9,                # Top-p (nucleus) sampling parameter - higher values make the output more diverse
            top_k=40                  # Top-k sampling parameter - higher values make the output less random
        )

        # Generate the content using the model - this will rank the message based on the CRITERIA scale
        print("Generating CRITERIA scores for the message...")
        response = gemini_model.generate_content(
            contents=[Content(role="user", parts=[Part.from_text(prompt)])],
            generation_config=generation_config
        )

        # Extract and parse the JSON response - this will extract the CRITERIA scores from the generated content
        criteria_scores = extract_json_from_response(response.text)
        if criteria_scores:
            return {
                "Cultural_Sensitivity": criteria_scores["CRITERIA_scores"]["Cultural_Sensitivity"],
                "Relevance": criteria_scores["CRITERIA_scores"]["Relevance"],
                "Insight": criteria_scores["CRITERIA_scores"]["Insight"],
                "Trustworthiness": criteria_scores["CRITERIA_scores"]["Trustworthiness"],
                "Emotional_Impact": criteria_scores["CRITERIA_scores"]["Emotional_Impact"],
                "Rationality": criteria_scores["CRITERIA_scores"]["Rationality"],
                "Influence": criteria_scores["CRITERIA_scores"]["Influence"],
                "Alignment": criteria_scores["CRITERIA_scores"]["Alignment"],
                "CRITERIA_final_score": criteria_scores["CRITERIA_final_score"]
            }
        else:
            print("No CRITERIA scores found in the response.")
            return None
    except Exception as e:
        print(f"An error occurred during message ranking: {e}")
        return None

def process_file(input_file, output_file, max_messages, start_message):
    """
    Process the input file, rank messages, and save the results to the output file.

    Args:
    input_file (str): Path to the input JSONL file.
    output_file (str): Path to the output JSONL file.
    max_messages (int): Maximum number of messages to process.
    start_message (int): Starting message number for processing.
    """
    print(f"Processing file: {input_file}")
    message_count = 0

    try:
        with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'a', encoding='utf-8') as outfile:
            # Skip lines until the start_message - useful for resuming processing
            for _ in range(start_message - 1):
                try:
                    next(infile)
                except StopIteration:
                    print("Reached the end of file while skipping lines.")
                    return

            for line in infile:
                if message_count >= max_messages:
                    print("Reached the maximum number of messages to process.")
                    break

                try:
                    message_tree = json.loads(line)
                except json.JSONDecodeError:
                    print("Error decoding JSON, skipping line.")
                    continue

                def process_message(message):
                    """
                    Recursively process a message and its replies.

                    Args:
                    message (dict): The message to process.
                    """
                    nonlocal message_count
                    if message_count >= max_messages:
                        return

                    print(f"Processing message {start_message + message_count}...")
                    criteria_scores = rank_message(message['text'])
                    if criteria_scores is not None:
                        message['CRITERIA'] = criteria_scores
                        print(f"CRITERIA scores added to message {start_message + message_count}.")
                    else:
                        print(f"Failed to add CRITERIA scores to message {start_message + message_count}.")

                    for reply in message.get('replies', []):
                        process_message(reply)

                    message_count += 1

                # Process the main message and its replies - this will rank the message and its replies
                process_message(message_tree['prompt'])
                for reply in message_tree['prompt'].get('replies', []):
                    process_message(reply)

                # Write the processed message to the output file - this will save the message with CRITERIA scores
                json.dump(message_tree, outfile, ensure_ascii=False)
                outfile.write('\n')
                print(f"Message {start_message + message_count} written to output file.")

            # Final save and download at the end of processing - this will save the final progress
            print("Final save...")
            files.download(output_file)
            print(f"File {output_file} downloaded successfully.")

    except Exception as e:
        print(f"An error occurred: {e}")

# Set definitions for all variables - adjust as needed
input_file = 'INPUT_FILE_PATH'  # Path to the input JSONL file - replace with your file path
output_file = 'OUTPUT_FILE_PATH'  # Path to the output JSONL file - replace with the name you want for the output file
max_messages = 5000  # Set the maximum number of messages to process - recommend starting with 5,000 or less to avoid timeouts
start_message = 1  # Starting message number for processing - adjust as needed to begin script from a specific message

# Process the file with a message limit - this will rank messages and save the results
process_file(input_file, output_file, max_messages, start_message)